In [8]:
#获得目录24nwp下的所有文件名
import os
def get_file_name(path):
    file_name = []
    for root, dirs, files in os.walk(path):
        for file in files:
            file_name.append(file)
    return file_name
files=get_file_name('24nwp')

In [9]:
#获得所有文件的大小
def get_file_size(path):
    file_size = []
    for root, dirs, files in os.walk(path):
        for file in files:
            file_size.append(os.path.getsize(os.path.join(root, file)))
    return file_size
file_size=get_file_size('24nwp')

In [10]:
#去除第一个文件和大小为0的文件
files_one=files[0]
files=files[1:]
file_size=file_size[1:]
files_new=[]
file_size_new=[]
files_new_riqi=[]
file_size_new_riqi=[]
for i in range(len(file_size)):
    if file_size[i]>=0.5:
        files_new_riqi.append(files[i])
        file_size_new_riqi.append(file_size[i])
        files_new.append(files[i])
        file_size_new.append(file_size[i])
#取除后缀外的后三个字符为文件名
for i in range(len(files_new)):
    files_new[i]=files_new[i][19:-3]
#将文件名和文件大小合并成一个字典
file_name_size=dict(zip(files_new,file_size_new))

In [11]:
#加载24nwp目录下的第一个文件
import json
data_pow=json.load(open('24nwp/'+files_one,'r'))

In [17]:
#读取files_new_riqi中的第一个文件，文件类型为rb
#偏码方式为'gbk'
import pandas as pd
data_cloud=pd.read_csv('24nwp/'+files_new_riqi[0],encoding='gbk',sep='	')

In [18]:
#data_pow转成dataframe格式
data_pow=pd.DataFrame(data_pow)

In [93]:
index_pow=0

In [20]:
#data_pow添加data_cloud的所有列的名字，用于后续的合并，初始化为0
for i in range(len(data_cloud.columns)):
    data_pow[data_cloud.columns[i]]=0

In [21]:
#去除data_pow的"@顺序","统一编码","时间"三列
data_pow=data_pow.drop(["@顺序","统一编码","时间"],axis=1)

In [94]:
for index_files in range(len(files_new_riqi)):
    data_cloud=pd.read_csv('24nwp/'+files_new_riqi[index_files],encoding='gbk',sep='	')
    data_pow=pd.DataFrame(data_pow)
    for index_cloud in range(len(data_cloud['@顺序'])):
        if(type(data_cloud['时间'][index_cloud])!=str):
            continue
        if(data_cloud['时间'][index_cloud]!='24:00:00'):
            time_cloud="2023-0"+files_new[index_files][0]+"-"+\
            files_new[index_files][1:3]+" "+files_new[index_files][3:5]+""+data_cloud['时间'][index_cloud]
        else:
            riqi=int(files_new[index_files][1:3])+1
            if(riqi<10):
                riqi=files_new[index_files][0]+"-"+"0"+str(riqi)
            elif(riqi<32):
                riqi=files_new[index_files][0]+"-"+str(riqi)
            else:
                riqi=str(int(files_new[index_files][0])+1)+"-01"
            time_cloud="2023-0"+riqi+" "+files_new[index_files][3:5]+"00:00:00"
        while data_pow['dateTime'][index_pow]!=time_cloud:
            index_pow+=1
        for i in range(len(data_cloud.columns)-3):
            data_pow[data_cloud.columns[i+3]][index_pow]=data_cloud[data_cloud.columns[i+3]][index_cloud]
        index_pow+=1

C:\Users\Administrator\AppData\Local\Temp\ipykernel_4192\419238181.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_pow[data_cloud.columns[i+3]][index_pow]=data_cloud[data_cloud.columns[i+3]][index_cloud]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_4192\419238181.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_pow[data_cloud.columns[i+3]][index_pow]=data_cloud[data_cloud.columns[i+3]][index_cloud]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_4192\419238181.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

KeyError: 178550

In [42]:
time_cloud="2023-0"+files_new[0][0]+"-"+\
    files_new[0][1:3]+" "+files_new[0][3:5]+""+data_cloud['时间'][index_cloud]

In [88]:
str(int(files_new[index_files][1:3]))

'900'

In [79]:
time_cloud

'2023-05-03 00:15:00'

In [82]:
data_pow['dateTime'][11580]==time_cloud

True

In [44]:
time_cloud="2023-0"+files_new[0][0]+"-"+\
        files_new[0][1]+str(int(files_new[0][2])+1)+" "+files_new[0][3:5]+""+data_cloud['时间'][index_cloud]

In [57]:
time_cloud="2023-0"+files_new[0][0]+"-"+\
        files_new[0][1:3]+" "+files_new[0][3:5]+""

In [69]:
#判断data_cloud['时间'][index_cloud]是否为空
nan=data_cloud['时间'][index_cloud]

In [74]:
type(data_cloud['时间'][index_cloud])==str

False

In [95]:
#遍历data_pow的所有行，当'环境温度','湿度','气压','风速','风向'都为0时，删除该行
for i in range(len(data_pow['气压'])):
    if(data_pow['环境温度'][i]==0 and data_pow['湿度'][i]==0 and data_pow['气压'][i]==0 and data_pow['风速'][i]==0 and data_pow['风向'][i]==0):
        data_pow=data_pow.drop([i],axis=0)

KeyboardInterrupt: 

In [103]:
#创建一个新的dataframe，列名称为data_pow的列名称
#data_pow刷新index，从0开始
data_pow=data_pow.reset_index()
data_pow_new=pd.DataFrame(columns=data_pow.columns)
for i in range(len(data_pow['气压'])):
    if(data_pow['环境温度'][i]==0 and data_pow['湿度'][i]==0 and data_pow['气压'][i]==0 and data_pow['风速'][i]==0 and data_pow['风向'][i]==0):
        continue
    else:
        data_pow_new=data_pow_new.append(data_pow.iloc[i])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_4192\2636668680.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pow_new=data_pow_new.append(data_pow.iloc[i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_4192\2636668680.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pow_new=data_pow_new.append(data_pow.iloc[i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_4192\2636668680.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pow_new=data_pow_new.append(data_pow.iloc[i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_4192\2636668680.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pow_ne

In [101]:
data_pow_1=data_pow.reset_index()

In [102]:
data_pow_1['湿度'][0]

72.68

In [104]:
#将data_pow_new导出为csv文件,编码格式为gbk
data_pow_new.to_csv('24nwp/24nwp.csv',encoding='gbk')